In [ ]:
import pandas as pd


# Investigate the Dataset

In [ ]:

# Load dataset
df = pd.read_csv("../data/task.csv")

# Display basic info
print("First 5 rows:")
print(df.head())

print("\nDataFrame Info:")
print(df.info())

print("\nLabel Distribution:")
print(df['Tag'].value_counts())



   Unnamed: 0                    Title  \
0           0  I tre volti della paura   
1           1        Mitt liv som hund   
2           2                The Brood   
3           3              The Haunted   
4           4        The Frozen Ground   

                                            Synopsis         Tag  
0  Note: this synopsis is for the orginal Italian...        cult  
1  The action takes place in the years 1958-1959 ...        cult  
2  At the Somafree Institute, Dr. Hal Raglan humi...        cult  
3  This creepy and scary story centers around The...  paranormal  
4  The film opens in an Anchorage motel room in 1...    dramatic  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566 entries, 0 to 1565
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1566 non-null   int64 
 1   Title       1566 non-null   object
 2   Synopsis    1566 non-null   object
 3   Tag         1566 non-null   object

In [4]:
from sklearn.preprocessing import LabelEncoder
import re

# Clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return text

df['text'] = df['text'].apply(clean_text)

# Encode tags
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['Tag'])


KeyError: 'text'